# Assignment 3

Turn in the assignment via the autograder:
`python3 -m autograder.run.submit Assignment3.ipynb` from the Assignment3 directory.

Before you turn this problem in, make sure everything runs as expected. To do so, restart the kernel and run all cells (in the menubar, select Runtime→→Restart and run all).
#### Show your work!
Whenever you are asked to find the solution to a problem, be sure to also **show how you arrived** at your answer.

### Resources

[1] https://community.plotly.com/t/two-3d-surface-with-different-color-map/18931/2 (Different color plots plotly)

[2] https://plotly.com/python/3d-surface-plots/ [plotting surfaces]

[3] https://community.plotly.com/t/3d-scatter-plot-with-surface-plot/27556/5 [plotting scatter plots]

[4] https://community.plotly.com/t/what-colorscales-are-available-in-plotly-and-which-are-the-default/2079 [color scales]

## Problem 1: Local Search on the Ackley Surface
---

As discussed in class with Local Search problems the path to the goal is not as important as the goal itself. In this question we work with the Ackley function, a non-convex function typically used as a test for optimization algorithms.

The Achkey function has many local optima but only one global optimum that is $f(0,0) = 0$. Interestingly enough it might not be that easy to find this solution.

The Ackley function is defined as:

$$f(x,y):= -20exp[-0.2\sqrt{0.5(x^2 + y^2)}] -exp[0.5(cos(2\pi x) + cos(2\pi y))] + e + 20$$

The figure below illustrates the result of running three optimization methods on the Ackley Function. These solutions were found using methods A, B and C.

<img src="images/ackley.png" alt="Ackley Function figure" width="400"/>

In this question, you will implement 3 optimization algorithms: 

1.   Stochastic Hill Climbing with Restarts (SHCR)
2.   Simulated Annealing (SA)
3.   Local Beam Search (LBS)

**Deliverables**

1. Complete this Notebook with implementation of SHCR, SA and LBS.
2. For each algorithm **visualize** the candidate solutions found at each
step using the provided ```create_ackley_figure()``` function. please see the image above an example of an image that should be included in your submission.



In [3]:
import numpy as np
from numpy import arange
from numpy import exp
from numpy import sqrt
from numpy import cos
from numpy import e
from numpy import pi
from numpy import meshgrid
from numpy.random import randn
from numpy.random import rand
from numpy.random import seed
from numpy.random import uniform
# hill climbing search of the ackley objective function
from numpy import asarray
import plotly.graph_objects as go

# objective function
def f(x, y):
	return -20.0 * exp(-0.2 * sqrt(0.5 * (x**2 + y**2))) - exp(0.5 * (cos(2 * pi * x) + cos(2 * pi * y))) + e + 20

def create_ackley_figure(solution_points=np.array([]), soln_names=['A','B,','C']):
  """
  :param solution_points A numpy array of dimensions [i, n, 2]
  where i is the number of solution points you want to plot and n is the 
  number of steps used in estimating your solution. In each case use the same 
  number of steps if you want to plot all solutions on the same figure.

  :return An interactive Ackley function figure with/without solution points.
  """
  # define range for input
  r_min, r_max = -5.0, 5.0
  # sample input range uniformly at 0.1 increments
  xaxis = arange(r_min, r_max, 0.1) # (100,)
  yaxis = arange(r_min, r_max, 0.1) # (100,)
  # create a mesh from the axis
  x, y = meshgrid(xaxis, yaxis)

  # compute targets
  results = f(x, y)

  figure = go.Figure(data=[go.Surface(z=results, x=x, y=y,colorscale='Blues',showscale=False)])
  figure.update_layout(title='Ackley Function', autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))

  if solution_points.size:
    soln_colors = ['Rainbow', 'Reds', 'Viridis']
    for i in range(len(solution_points)):
      x_sln = solution_points[i][:,0]
      y_sln = solution_points[i][:,1]
      zdata = f(x_sln, y_sln)
      figure.add_scatter3d(name=soln_names[i], x=x_sln, y=y_sln, z=zdata, mode='markers',
      marker=dict(size=1, color=[i / len(x_sln) for i in range(len(x_sln))], colorscale=soln_colors[i]))
      
      figure.update_layout(showlegend=True)
      
  return figure

In [4]:
### RUN THIS ###
"""
This is the Ackley Function.
"""
ack_figure = create_ackley_figure()
ack_figure

### General Function Definitions

In [5]:
# objective function
def objective(v):
  """
  This function defines the Ackley surface, it has a minimum of 0. This 
  can be used to test if we have found points that yield the minimum value.
  :param v, a tuple representing a 2D point being tested.

  returns a value in the range [-5.0, 5.0]
  """
  x, y = v
  return -20.0 * exp(-0.2 * sqrt(0.5 * (x**2 + y**2))) - exp(0.5 * (cos(2 * pi * x) + cos(2 * pi * y))) + e + 20

  
# check if a point is within the bounds of the search
def in_bounds(point, bounds=asarray([[-5.0, 5.0], [-5.0, 5.0]])):
  """
  It is possible our optimzation method could move us outside of our search 
  space, so it is helpful to check.
  :param point a tuple representing a 2D point
  :param a 2D array, that describes the bounds of each dimension of our point.
  returns a Boolean of whether the point lies in the bounds or not.
  """
	# enumerate all dimensions of the point
  for d in range(len(bounds)):
    # check if out of bounds for this dimension
    if point[d] < bounds[d, 0] or point[d] > bounds[d, 1]:
      return False
  return True

In [6]:
# Code for generating starting points
def generate_starting_points(bounds):
    start_position = rand(2)
    start_position[0] = start_position[0] * (bounds[0][-1] - bounds[0][0]) + bounds[0][0]
    start_position[1] = start_position[1] * (bounds[1][-1] - bounds[1][0]) + bounds[1][0]
    return start_position

# Code for generating stochastic neighbors
def generate_stochastic_neighbor(current_point, bounds, step_size):
    return current_point + random.uniform(low = -1, high = 1, size = len(bounds)) * step_size

### Part 1) Stochastic Hill Climbing with Restarts

1.   Implement SHCR in the cell below.
2.   Visualize and comment on the candidate points visited.


In [7]:
# stochastic hill climbing with restarts algorithm
def stochastic_hillclimbing(objective, bounds, n_iterations, step_size, start_pt):
  """
  :param objective, the function we are trying to optimize
  :param bounds, the boundaries of the problem
  :param n_iterations number of times to repeat stochastic hill climbing
  :param step_size how much we should move
  :param start_pt the point we start optimizing from.
  returns [solution, solution_value, candidates]
  """
  current = np.asarray(start_pt, dtype=float)
  current_eval = objective(current)
  candidates = [current.copy()]
  for _ in range(n_iterations):
    candidate = current + uniform(low=-1, high=1, size=len(bounds)) * step_size
    if in_bounds(candidate, bounds):
      candidate_eval = objective(candidate)
      if candidate_eval < current_eval:
        current, current_eval = candidate, candidate_eval
    candidates.append(current.copy())
  return current, current_eval, np.asarray(candidates)

def random_restarts(objective, bounds, n_iter, step_size, n_restarts):
  """
  :param objective, the function we are trying to optimize
  :param bounds, the boundaries of the problem
  :param n_iter number of times to repeat stochastic hill climbing 
  :param step_size how much we should move
  :param n_restarts, the number of times we restart.
  returns [best solution, best solution value, visited points]
  """
  best, best_eval = None, float("inf")
  all_candidates = []
  for _ in range(n_restarts):
    start_pt = generate_starting_points(bounds)
    solution, solution_eval, candidates = stochastic_hillclimbing(
      objective, bounds, n_iter, step_size, start_pt
    )
    all_candidates.append(candidates)
    if solution_eval < best_eval:
      best, best_eval = solution, solution_eval
  visited_points = np.concatenate(all_candidates, axis=0) if all_candidates else np.array([])
  return best, best_eval, visited_points


# TEST CASE:

# seed the pseudorandom number generator
seed(240)
# define range for input
bounds = asarray([[-5.0, 5.0], [-5.0, 5.0]])
# define the total iterations
n_iter = 100
# define the maximum step size
step_size = 0.05
# total number of random restarts
n_restarts = 300
# perform the hill climbing search
best, score, shcr_candidates = random_restarts(objective, bounds, n_iter, step_size, n_restarts)
print('Done!')
print('f(%s) = %f' % (best, score))

Done!
f([-0.00194521 -0.00151671]) = 0.007139


In [8]:
# Helpful analysis
# An example of plotting solutions from 3 optimziation methods.
# all_method_candidates has shape [3, 1000, 2], the first dim is the number of solutions
# the second dim is the number of steps taken in each method.
shcr_candidates = np.random.rand(100,2) # EXAMPLE PURPOSES ONLY replace with your actual solution.
shcr_candidates = shcr_candidates.reshape(-1,2)
all_method_candidates = np.array([shcr_candidates])
ack_figure = create_ackley_figure(all_method_candidates)
#ack_figure

### Optional Analysis - Stochastic Hill Climbing


In [9]:
# OPTIONAL: Analysis to graph your answers; comment out once stochastic_hillclimbing is implemented

all_method_candidates = np.array([shcr_candidates])
ack_figure = create_ackley_figure(all_method_candidates)
#ack_figure.show()


The figure depicts each search path for the given random restarts done for the stochastic hill algorithm. Eventually, after a certain number of restarts, we choose an initial point close enought to the optimum at f(0,0) so that SHC was able to find the point. This demonstrates how SHC with restarts is asymtotically complete, as it will evenutally reach the global minimum after a asymptotic number of random restarts.

### Part 2) Simulated Annealing

1.   Implement SA in the cell below.
2.   Visualize and comment on the candidate points visited.

In [10]:
def get_temperature_schedule(epoch, temp, cos_frequency, diff):
  """
  :param epoch Iterations elapsed
  :param temp Temperature
  :param cos_frequency Cosine annealing cycle frequency
  :param diff Difference between value of candidate solution and current solution.
  """
  # calculate temperature for current epoch (1.001 to prevent t from going to zero)
  t = temp * (1.001 + np.cos(epoch*(2*np.pi*cos_frequency)))/2
  # calculate acceptance criterion
  criterion = exp(-diff / t)
  return criterion

# simulated annealing algorithm
def simulated_annealing(objective, bounds, n_iterations, step_size, temp, cos_frequency):
  """
  :param objective, the function we are trying to optimize
  :param bounds, the boundaries of the problem
  :param n_iterations number of times to repeat stochastic hill climbing
  :param step_size how much we should move
  :param temp temperature for each epoch
  :param cos_frequency Cosine annealing cycle frequency
  returns [solution, solution_value, candidates]
  """
  current = generate_starting_points(bounds)
  current_eval = objective(current)
  best, best_eval = current.copy(), current_eval
  candidates = [current.copy()]
  for epoch in range(n_iterations):
    candidate = current + uniform(low=-1, high=1, size=len(bounds)) * step_size
    if not in_bounds(candidate, bounds):
      candidates.append(current.copy())
      continue
    candidate_eval = objective(candidate)
    diff = candidate_eval - current_eval
    if diff < 0:
      current, current_eval = candidate, candidate_eval
    else:
      criterion = get_temperature_schedule(epoch, temp, cos_frequency, diff)
      if rand() < criterion:
        current, current_eval = candidate, candidate_eval
    if current_eval < best_eval:
      best, best_eval = current.copy(), current_eval
    candidates.append(current.copy())
  return best, best_eval, np.asarray(candidates)

# seed the pseudorandom number generator
seed(240)
# define range for input
bounds = asarray([[-5.0, 5.0], [-5.0, 5.0]])
# define the total iterations
n_iterations = 10000
# define the maximum step size
step_size = 0.1
# initial temperature
temp = 5
# cosine annealing cycle frequency
cos_frequency = .01
# perform the simulated annealing search
best, score, sa_candidates = simulated_annealing(objective, bounds, n_iterations, step_size, temp, cos_frequency)
print('Done!')
print('f(%s) = %f' % (best, score))

Done!
f([-3.32994829e-03  6.85092128e-05]) = 0.009716


### Optional Analysis - Simulated annealing

In [11]:
# OPTIONAL: Analysis to graph your answers; comment out once simulated_annealing is implemented

print(sa_candidates)
all_method_candidates = np.array([sa_candidates])
ack_figure = create_ackley_figure(all_method_candidates)
#ack_figure.show()

[[-2.0649346  -0.69623134]
 [-2.00818803 -0.74744879]
 [-2.04179104 -0.68889041]
 ...
 [ 0.30796024  0.71931967]
 [ 0.24161829  0.72869314]
 [ 0.16296644  0.78924605]]


### Part 3) Local Beam Search



1.   Implement LBS in the cell below.
2.   Visualize and comment on the candidate points visited.
3.   Compare all 3 implementations by commenting on the distribution of points on the Ackley surface and the empirical run time of each method.


In [12]:
# generates a cross shaped set of neighbors based on step size
def generate_neighbors(point, step_size=0.1):
    point = point.reshape(2,1)
    shifts = np.array([step_size*(2**i) for i in range(0,11)])
    neighbors = point + np.concatenate([
       np.array([np.zeros(len(shifts)),  shifts]),
       np.array([np.zeros(len(shifts)), -shifts]),
       np.array([ shifts, np.zeros(len(shifts))]),
       np.array([-shifts, np.zeros(len(shifts))]),
    ], axis=1)
    return neighbors

def local_beam_search(objective, bounds, step_size, k, n_iterations):
  """
  :param objective, the function we are trying to optimize
  :param bounds, the boundaries of the problem
  :param k, how many candidates to consider
  :param n_iterations, how long to search for.
  returns [solution, solution_value, candidates]
  """
  if k <= 0:
    return None, float("inf"), np.array([])

  beams = [np.asarray(generate_starting_points(bounds), dtype=float) for _ in range(k)]
  visited = []
  best, best_eval = None, float("inf")
  for point in beams:
    visited.append(point.copy())
    value = objective(point)
    if value < best_eval:
      best, best_eval = point.copy(), value

  for _ in range(n_iterations):
    all_neighbors = []
    for point in beams:
      neighbors = generate_neighbors(point, step_size).T
      for neighbor in neighbors:
        if in_bounds(neighbor, bounds):
          all_neighbors.append(neighbor)

    if not all_neighbors:
      break

    scores = [objective(candidate) for candidate in all_neighbors]
    order = np.argsort(scores)
    beams = [np.asarray(all_neighbors[i], dtype=float) for i in order[:k]]

    for point in beams:
      visited.append(point.copy())
    if scores[order[0]] < best_eval:
      best, best_eval = beams[0].copy(), scores[order[0]]

  return best, best_eval, np.asarray(visited)

# seed the pseudorandom number generator
seed(240)
# define range for input
bounds = asarray([[-5.0, 5.0], [-5.0, 5.0]])
# define the total iterations
n_iterations = 1000
# define the maximum step size
step_size = 0.002
# candidates to consider
k = 3
solution, solution_value, lbs_candidates = local_beam_search(objective, bounds, step_size, k, n_iterations)

In [13]:
# OPTIONAL: Analysis to graph your answers; comment out once local_beam_search is implemented

all_method_candidates = np.array([lbs_candidates])
ack_figure = create_ackley_figure(all_method_candidates)
ack_figure.show()

In [14]:
# Optional: Analysis of all Three Experiment Graphs

min_size = min(len(shcr_candidates), len(sa_candidates), len(lbs_candidates))

all_method_candidates = np.array([
    shcr_candidates[0:min_size],
    sa_candidates[0:min_size],
    lbs_candidates[0:min_size],
])
ack_figure = create_ackley_figure(all_method_candidates)
#ack_figure.show()


## Problem 2: CSP
---

Consider the following constraint satisfaction problem. A linear graph has nodes of the following colors:

- Red
- Yellow
- Green
- Blue
- Violet

Each node has a domain of {1, 2, ..., 9}.

Each node type has the following constraints on its value:

- Red - No contraints
- Yellow - equals the rightmost digit of of the product of all its neighbors
- Green - equals the rightmost digit of the sum of all its neighbors 
- Blue - equals the leftmost digit of the sum of all its neighbors
- Violet - equals the leftmost digit of the product of all of its neighbors

As a reminder here is the pseudo code for the Min-Conflicts search algorithm:
<img src="images/min_conflicts.png" alt="min conflicts algorithms" width="400"/>

**Notes:**

- It's possible that you won't converge to a solution in a single run. Try a few runs to see if you get to a solution.
- The testcases are to show you what a problem looks like, we will test/grade your program on different examples.

### Part 1) Implementation
Complete the function ```solve_csp``` defined below. You may find some helper functions useful.

In [1]:
def solve_csp(nodes, arcs, max_steps):
    """
    This function solves the csp using the MinConflicts Search Algorithm.

    :param nodes, a list of letters that indicates what type of node it is,
                  the index of the node in the list indicates its id
                  letters = {R, Y, G, B, V}

    :param arcs,  a list of tuples that contains two numbers, indicating the
                  IDs of the nodes the arc connects.

    :param max_steps, max number of steps to make before giving up

    returns a list of values for the solution to the CSP where the
             index of the value corresponds the value for that given node.
    """
    n = len(nodes)
    if n == 0:
        return []

    neighbors = [[] for _ in range(n)]
    for a, b in arcs:
        if 0 <= a < n and 0 <= b < n:
            neighbors[a].append(b)
            neighbors[b].append(a)

    def leftmost_digit(value):
        value = abs(int(value))
        return int(str(value)[0]) if value != 0 else 0

    def expected_value(color, neighbor_vals):
        if color == "R":
            return None
        if not neighbor_vals:
            return 0
        total = sum(neighbor_vals)
        prod = 1
        for v in neighbor_vals:
            prod *= v
        if color == "Y":
            return prod % 10
        if color == "G":
            return total % 10
        if color == "B":
            return leftmost_digit(total)
        if color == "V":
            return leftmost_digit(prod)
        return None

    def is_satisfied(idx, assignment):
        color = nodes[idx]
        if color == "R":
            return True
        neighbor_vals = [assignment[j] for j in neighbors[idx]]
        return assignment[idx] == expected_value(color, neighbor_vals)

    def total_conflicts(assignment):
        return sum(0 if is_satisfied(i, assignment) else 1 for i in range(n))

    assignment = [int(np.random.randint(1, 10)) for _ in range(n)]

    for _ in range(max_steps):
        conflicted = [i for i in range(n) if not is_satisfied(i, assignment)]
        if not conflicted:
            return assignment

        var = int(np.random.choice(conflicted))
        best_values = []
        best_score = None
        original = assignment[var]
        for val in range(1, 10):
            assignment[var] = val
            score = total_conflicts(assignment)
            if best_score is None or score < best_score:
                best_score = score
                best_values = [val]
            elif score == best_score:
                best_values.append(val)
        assignment[var] = int(np.random.choice(best_values)) if best_values else original

    return []


### Part 2) Test Cases

Below we've included 4 test cases to help you debug your code. Your submitted code will be tested on other cases as well, but if your implementation of the above Min-Conflicts search algorithm is able to solve these problems, you should be good to go.

In [15]:
# test Case 1

nodes = 'YGVRB'
arcs = [(0,1), (0,2), (1,2), (1,3), (1,4), (2,3), (2,4)]
max_steps = 1000

for _ in range(max_steps):
    sol = solve_csp(nodes, arcs, max_steps)
    if sol != []:
        print(nodes)
        break
        
all_solutions = [[1, 1, 1, 7, 2],[2, 1, 2, 4, 3],[2, 6, 7, 6, 1],[2, 8, 9, 6, 1],
                 [3, 3, 1, 5, 4],[6, 2, 8, 7, 1],[6, 7, 8, 2, 1],[6, 9, 4, 8, 1]]

if sol == []:
    print('No solution')
else:
    if sol in all_solutions:
        print('Solution found:', sol)
    else:
        print('ERROR: False solution found:', sol)

YGVRB
Solution found: [6, 2, 8, 7, 1]


In [16]:
# test Case 2

nodes = 'YVBGR'
arcs = [(0,1), (0,2), (1,3), (2,4)]
max_steps = 1000

for _ in range(max_steps):
    sol = solve_csp(nodes, arcs, max_steps)
    if sol != []:
        print(nodes)
        break
        
all_solutions = [[1, 1, 1, 1, 9], [1, 3, 7, 3, 6], [1, 7, 3, 7, 2], [1, 9, 9, 9, 8]]

if sol == []:
    print('No solution')
else:
    if sol in all_solutions:
        print('Solution found:', sol)
    else:
        print('ERROR: False solution found:', sol)

YVBGR
Solution found: [1, 3, 7, 3, 6]


In [17]:
# test Case 3

nodes = 'VYGBR'
arcs = [(0,1), (1,2), (2,3), (3,4)]
max_steps = 1000

for _ in range(max_steps):
    sol = solve_csp(nodes, arcs, max_steps)
    if sol != []:
        print(nodes)
        break
        
all_solutions = [[2, 2, 1, 9, 8],[3, 3, 1, 8, 7],[4, 4, 1, 7, 6],[5, 5, 1, 6, 5],[5, 5, 3, 8, 5],
                 [6, 6, 1, 5, 4],[7, 7, 1, 4, 3],[8, 8, 1, 3, 2],[8, 8, 6, 8, 2],[9, 9, 1, 2, 1]]

if sol == []:
    print('No solution')
else:
    if sol in all_solutions:
        print('Solution found:', sol)
    else:
        print('ERROR: False solution found:', sol)

VYGBR
Solution found: [8, 8, 1, 3, 2]


In [18]:
# test Case 4

nodes = 'YGVBR'
arcs = [(0,1), (0,2), (1,3), (2,3), (3,4), (1,2)]
max_steps = 1000

for _ in range(max_steps):
    sol = solve_csp(nodes, arcs, max_steps)
    if sol != []:
        print(nodes)
        break
        
all_solutions = [[4, 4, 1, 9, 4],[4, 7, 2, 1, 1],[4, 7, 2, 1, 2],[4, 7, 2, 1, 3],[4, 7, 2, 1, 4],
                 [4, 7, 2, 1, 5],[4, 7, 2, 1, 6],[4, 7, 2, 1, 7],[4, 7, 2, 1, 8],[4, 7, 2, 1, 9],
                 [4, 8, 3, 1, 1],[4, 8, 3, 1, 2],[4, 8, 3, 1, 3],[4, 8, 3, 1, 4],[4, 8, 3, 1, 5],
                 [4, 8, 3, 1, 6],[4, 8, 3, 1, 7],[4, 8, 3, 1, 8],[5, 1, 5, 1, 4],[5, 1, 5, 1, 5],
                 [5, 1, 5, 1, 6],[5, 1, 5, 1, 7],[5, 1, 5, 1, 8],[5, 1, 5, 1, 9]]

if sol == []:
    print('No solution')
else:
    if sol in all_solutions:
        print('Solution found:', sol)
    else:
        print('ERROR: False solution found:', sol)

YGVBR
Solution found: [5, 1, 5, 1, 4]
